Code to extract numerical and time series data.

Things to note:

Because the chartevents data is very large:

1. Only 20,000,000 rows were loaded and among these rows
2. For each hadm_id only the first reading for each item was extracted

In [63]:
# Import libraries
import os
import pandas as pd

In [64]:
# Load the data

#items 
items = pd.read_csv(os.path.join('..','d_items.csv'), usecols= ['itemid', 'label', 'abbreviation'])

items = items[items['itemid'].isin([220210, 220277, 225309, 220045, 220739, 223900, 223901, 223762])]


In [65]:
# chartevents
chart = pd.read_csv(os.path.join('..','chartevents.csv'), usecols= ['subject_id', 'hadm_id', 'charttime','itemid', 'valuenum', 'valueuom'], nrows=20000000)

chart = chart[chart['itemid'].isin([220210, 220277, 225309, 220045, 220739, 223900, 223901, 223762])]

chart = chart.groupby(['hadm_id', 'itemid']).first().reset_index()

#chart.head()

In [66]:
# Combining the dataset
num_data = pd.merge(chart, items, on = ['itemid'])

#num_data.head()

In [67]:
# Use the items as columns
num_data = num_data.pivot_table(index= ['subject_id', 'hadm_id', 'charttime'], columns = 'label', values = 'valuenum').reset_index()

#num_data.head(10)

In [68]:
# Combine all the GCS to get the total
num_data['GCS Total'] = num_data[['GCS - Eye Opening', 'GCS - Motor Response', 'GCS - Verbal Response']].sum(axis=1)

num_data = num_data.drop(columns = ['GCS - Eye Opening', 'GCS - Motor Response', 'GCS - Verbal Response'])

#num_data.sample(10)

In [69]:
# Get the age column
patient_data = pd.read_csv(os.path.join('..','patients.csv', 'patients.csv'), usecols= ['subject_id','anchor_age'])

#patient_data.head()

In [70]:
# Merge the age column to the numerical dataset
final = pd.merge(patient_data, num_data, on = ['subject_id'])

#final.head()

In [71]:
final.to_csv('NumericalData.csv')